## Work
1. 請嘗試將 preproc_x 替換成以每筆資料的 min/max 進行標準化至 -1 ~ 1 間，再進行訓練
2. 請嘗試將 mlp 疊更深 (e.g 5~10 層)，進行訓練後觀察 learning curve 的走勢
3. (optional) 請改用 GPU 進行訓練 (如果你有 GPU 的話)，比較使用 CPU 與 GPU 的訓練速度

In [1]:
## 
"""
Your code here (optional)
確認硬體資源
"""
#Not using NVIDA gpu
!nvidia-smi

'nvidia-smi' 不是內部或外部命令、可執行的程式或批次檔。


In [2]:
import os
import keras
from sklearn.preprocessing import StandardScaler
import numpy as np

# 請嘗試設定 GPU：os.environ
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [3]:
train, test = keras.datasets.cifar10.load_data()

In [4]:
## 資料前處理
"""
Your code here
"""
def preproc_x(x, flatten = True):
    if flatten == True:
        x = x.reshape((len(x), -1))
    #normalize to -1 & 1
    xmin = np.min(x)
    xmax = np.max(x)
    x = -1 + 2.0 * (x - xmin) / (xmax - xmin)
    return x

def preproc_y(y,num_classes = 10):
    y = keras.utils.to_categorical(y,num_classes)
    return y

In [5]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [6]:
"""Code Here
建立你的神經網路
"""

def build_mlp(input_shape, output_units = 10, num_neurons = [512,512,256,256,128,128]):
    
    input_layer = keras.layers.Input(input_shape)
    for i,n_units in enumerate(num_neurons):
        
        if i == 0:
            x = keras.layers.Dense(units = n_units, activation = 'relu')(input_layer)
        else:
            x = keras.layers.Dense(units = n_units, activation = 'relu')(x)
    
    output_layer = keras.layers.Dense(units = output_units, activation = 'softmax')(x)
    model = keras.models.Model(inputs = [input_layer], outputs = [output_layer])
    
    return model

In [7]:
model = build_mlp(x_train.shape[1:])
model.summary()

W1117 17:55:26.424571  6332 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1117 17:55:28.918416  6332 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1117 17:55:29.442164  6332 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_6 (Dense)              (None, 128)               16512     
__________

In [8]:
## 超參數設定
LEARNING_RATE = 0.001
EPOCHS = 20
BATCH_SIZE = 128

In [9]:
optimizer = keras.optimizers.Adam(lr=LEARNING_RATE)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

W1117 17:55:30.512837  6332 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1117 17:55:30.635419  6332 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [10]:
model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          validation_data=(x_test, y_test), 
          shuffle=True)

W1117 17:55:32.220560  6332 deprecation.py:323] From D:\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1117 17:55:32.281520  6332 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 21s 423us/step - loss: 1.7045 - acc: 0.3899 - val_loss: 1.5152 - val_acc: 0.4612
Epoch 2/20
50000/50000 [==============================] - 16s 314us/step - loss: 1.4764 - acc: 0.4745 - val_loss: 1.4398 - val_acc: 0.4833
Epoch 3/20
50000/50000 [==============================] - 16s 315us/step - loss: 1.3559 - acc: 0.5176 - val_loss: 1.4029 - val_acc: 0.5079
Epoch 4/20
50000/50000 [==============================] - 16s 327us/step - loss: 1.2534 - acc: 0.5564 - val_loss: 1.3813 - val_acc: 0.5188
Epoch 5/20
50000/50000 [==============================] - 17s 330us/step - loss: 1.1732 - acc: 0.5844 - val_loss: 1.3743 - val_acc: 0.5259
Epoch 6/20
50000/50000 [==============================] - 17s 338us/step - loss: 1.0925 - acc: 0.6135 - val_loss: 1.3774 - val_acc: 0.5288
Epoch 7/20
50000/50000 [==============================] - 17s 339us/step - loss: 1.0140 - acc: 0.6378 - val_loss: 1.

In [11]:
import matplotlib.pyplot as plt


train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]

train_acc = model.history.history["acc"]
valid_acc = model.history.history["val_acc"]

plt.plot(range(len(train_loss)), train_loss, label="train loss")
plt.plot(range(len(valid_loss)), valid_loss, label="valid loss")
plt.legend()
plt.title("Loss")
plt.show()

plt.plot(range(len(train_acc)), train_acc, label="train accuracy")
plt.plot(range(len(valid_acc)), valid_acc, label="valid accuracy")
plt.legend()
plt.title("Accuracy")
plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>